In [1]:
#!pip install streamlit

In [2]:
#!pip install streamlit pyngrok

In [3]:
!ngrok authtoken 30ExLK2yDH73ImYhMUlfbrV518q_87wDqbXdQGMkeiis6A7wV

Authtoken saved to configuration file: C:\Users\uttam\AppData\Local/ngrok/ngrok.yml


In [4]:
# Load Data and Models
#@st.cache_data(allow_output_mutation=True)
def load_models_and_data():
    # Load customer clustering model
    kmeans = joblib.load("kmeans_model.pkl")
    scaler = joblib.load("scaler.pkl")
    
    # Load pivot table for product similarity
    pivot = pd.read_pickle("product_customer_pivot.pkl")
    product_similarity = pd.read_pickle("product_similarity.pkl")

    return kmeans, scaler, pivot, product_similarity


In [29]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics.pairwise import cosine_similarity

#kmeans, scaler, pivot, product_similarity = load_models_and_data()
kmeans = joblib.load("kmeans_model.pkl")
scaler = joblib.load("scaler.pkl")
    
# Load pivot table for product similarity
pivot = pd.read_pickle("product_customer_pivot.pkl")
product_similarity = pd.read_pickle("product_similarity.pkl")

st.set_page_config(page_title="🛒 Shopper Spectrum App", layout="centered")
st.title("🛍️ Shopper Spectrum: E-Commerce Intelligence")
st.markdown("---")

# Tabs for App Modules
app_mode = st.sidebar.selectbox("Choose Module", ["📦 Product Recommendation", "👥 Customer Segmentation"])
# ====================================
# Module 1: Product Recommendation
# ====================================
if app_mode == "📦 Product Recommendation":
    st.subheader("🔍 Recommend Similar Products")

    product_input = st.text_input("Enter a Product Name:")

    if st.button("Get Recommendations"):
        if product_input in pivot.index:
            product_vector = product_similarity.loc[product_input].sort_values(ascending=False)[1:6]
            st.success("Recommended Products:")
            for i, product in enumerate(product_vector.index):
                st.markdown(f"{i+1}. **{product}**")
        else:
            st.error("Product not found. Please try another product name.")
# ====================================
# Module 2: Customer Segmentation
# ====================================
elif app_mode == "👥 Customer Segmentation":
    st.subheader("🧮 Predict Customer Segment")

    recency = st.number_input("Recency (days since last purchase):", min_value=0)
    frequency = st.number_input("Frequency (total transactions):", min_value=0)
    monetary = st.number_input("Monetary (total spend):", min_value=0.0)

    if st.button("Predict Cluster"):
        input_df = pd.DataFrame({"Recency": [recency], "Frequency": [frequency], "Monetory": [monetary]})
        scaled_input = scaler.transform(input_df)
        cluster_label = kmeans.predict(scaled_input)[0]

        segment_map = {
            0: "High-Value",
            1: "Regular",
            2: "Occasional",
            3: "At-Risk"
        }

        segment_name = segment_map.get(cluster_label, "Unknown")
        st.success(f"Customer Segment: **{segment_name}**")

st.markdown("---")
st.caption("📊 Powered by RFM Clustering & Collaborative Filtering")


Overwriting app.py


In [6]:
!ngrok authtoken 30ExLK2yDH73ImYhMUlfbrV518q_87wDqbXdQGMkeiis6A7wV

Authtoken saved to configuration file: C:\Users\uttam\AppData\Local/ngrok/ngrok.yml


In [25]:
# write code for Generating App link to streamlit
import os
import threading

def run_streamlit():
    os.system("streamlit run app.py --server.port 8080")

thread = threading.Thread(target = run_streamlit)
thread.start()

In [26]:
from pyngrok import ngrok
import time

time.sleep(5)

public_url = ngrok.connect(8080)
print('Your streamlit URL is :- ', public_url)

Your streamlit URL is :-  NgrokTunnel: "https://4f89feb15f11.ngrok-free.app" -> "http://localhost:8080"


In [27]:
import pandas as pd
import numpy as np
pivot = pd.read_pickle("product_customer_pivot.pkl")

In [28]:
print(pivot.index.tolist()[:10])  # Shows first 10 product names

[' 4 PURPLE FLOCK DINNER CANDLES', " 50'S CHRISTMAS GIFT BAG LARGE", ' DOLLY GIRL BEAKER', ' I LOVE LONDON MINI BACKPACK', ' I LOVE LONDON MINI RUCKSACK', ' NINE DRAWER OFFICE TIDY', ' OVAL WALL MIRROR DIAMANTE ', ' RED SPOT GIFT BAG LARGE', ' SET 2 TEA TOWELS I LOVE LONDON ', ' SPACEBOY BABY GIFT SET']
